In [327]:
import pandas as pd
import numpy as np
import itertools

In [328]:
men=["Anthony","Pratt","Nathan","Brandon","Alex","Layton","Curtis","John","Dario","Garland"]
women=["Jasmine","Paris","Ellie","Briana","Jenni","Christina","Tyler","Jessica","Alexandria","Ashley","Shelby"]

In [329]:
perfect_match = [("Curtis","Shelby"),("Pratt","Paris"),("John","Jenni"),("Alex","Jasmine"),("Dario","Ashley"),("Layton","Christina"),("")]
no_match = [("Brandon","Jessica"),("Brandon","Christina"),("Brandon","Alexandria"),("John","Jasmine"),("Nathan","Christina"),("Dario","Christina")]

In [330]:
def remove_matched (couples_list) :
    couples_list = [i for i in couples_list if i[0] not in [i[0] for i in perfect_match]]
    couples_list = [i for i in couples_list if i[1] not in [i[1] for i in perfect_match]]
    for i in perfect_match :
        couples_list.append(i)
    return couples_list

def match_remove_matched (couples_list) :
    couples_list = [i for i in couples_list if i[0] not in [i[0] for i in perfect_match]]
    couples_list = [i for i in couples_list if i[1] not in [i[1] for i in perfect_match]]
    return couples_list
    
def remove_no_match (couples_list) :
    couples_list = [i for i in couples_list if i not in [i for i in no_match]]
    return couples_list

In [331]:
match1 = [("Layton","Jenni"),("Dario","Ashley"),("Pratt","Paris"),("Curtis","Briana"),("John","Jasmine"),("Brandon","Christina"),("Garland","Alexandria"),("Anthony","Jessica"),("Nathan","Shelby"),("Alex","Ellie")]
match1_matches = 2
match2 = [("Layton","Jessica"),("Dario","Shelby"),("Pratt","Paris"),("Garland","Jasmine"),("John","Ashley"),("Brandon","Tyler"),("Alex","Christina"),("Anthony","Alexandria"),("Nathan","Jenni"),("Curtis","Briana")]
match2_matches = 2
match3 = [("John","Christina"),("Curtis","Briana"),("Nathan","Jessica"),("Brandon","Shelby"),("Pratt","Paris"),("Garland","Alexandria"),("Dario","Ashley"),("Alex","Jasmine"),("Anthony","Jenni"),("Layton","Ellie")]
match3_matches = 3
match4 = [("Brandon","Ellie"),("Anthony","Jenni"),("Curtis","Briana"),("Alex","Tyler"),("Dario","Shelby"),("Nathan","Alexandria"),("John","Christina"),("Garland","Jasmine"),("Layton","Jessica"),("Pratt","Paris")]
match4_matches = 1
match5 = [("Curtis","Shelby"),("Pratt","Paris"),("Anthony","Ashley"),("Brandon","Jasmine"),("Garland","Tyler"),("John","Jenni"),("Layton","Briana"),("Alex","Christina"),("Nathan","Jessica"),("Dario","Alexandria")]
match5_matches = 3
match6 = [("Curtis","Shelby"),("Pratt","Paris"),("Brandon","Ashley"),("Nathan","Jasmine"),("Dario","Briana"),("Anthony","Ellie"),("Garland","Tyler"),("John","Alexandria"),("Layton","Jenni"),("Alex","Jessica")]
match6_matches = 2
match7 = [("Curtis","Shelby"),("Pratt","Paris"),("John","Jenni"),("Dario","Ashley"),("Alex","Jasmine"),("Layton","Ellie"),("Brandon","Tyler"),("Anthony","Briana"),("Nathan","Christina"),("Garland","Jessica")]
match7_matches = 6
match8 = [("Curtis","Shelby"),("Pratt","Paris"),("John","Jenni"),("Nathan","Briana"),("Alex","Jasmine"),("Anthony","Jessica"),("Brandon","Tyler"),("Garland","Alexandria"),("Layton","Ellie"),("Dario","Ashley")]
match8_matches = 5
match9 = [("Curtis","Shelby"),("Pratt","Paris"),("John","Jenni"),("Alex","Jasmine"),("Nathan","Ellie"),("Garland","Jessica"),("Layton","Ashley"),("Dario","Ashley"),("Brandon","Briana"),("Anthony","Alexandria")]
match9_matches = 8
match10 = [("Curtis","Shelby"),("Pratt","Paris"),("John","Jenni")]
match10_matches = 3

In [332]:
def process_match (match, match_matches) :
    # remove perfect matches and reduce _matches by that value
    match_perfect_matches = [i for i in match if "".join(i) in ["".join(i) for i in perfect_match]]
    # if the remaining value is 0, add all matches to no_match
    if len(match_perfect_matches) == match_matches :
        match = [i for i in match if "".join(i) not in ["".join(i) for i in perfect_match]]
        for i in match :
            if "".join(i) not in ["".join(i) for i in no_match] :
                no_match.append(i)
            else :
                continue
        match_matches_remaining = match_matches - len(match_perfect_matches)
    else : 
        match_matches_remaining = match_matches - len(match_perfect_matches)
    match = remove_no_match(match_remove_matched(match))
    return (match, match_matches_remaining)


In [333]:
for n in reversed(range(1,11)) :
    exec("(match{n}, match{n}_matches) = process_match(match{n}, match{n}_matches)".format(n=n))
for n in range(1,11) :
    exec("(match{n}, match{n}_matches) = process_match(match{n}, match{n}_matches)".format(n=n))

In [334]:
def df_match_probability (match, match_matches) :
    df = pd.DataFrame(data=match, columns=["man","woman"])
    if match_matches == 0 :
        df["probability"] = 0
    else :
        df["probability"] = match_matches/len(match)
    return df

In [335]:
for n in range(1,11) :
    exec("match{n}_df = df_match_probability (match{n}, match{n}_matches)".format(n=n))

In [336]:
all_couples = list(product(men, women))
remaining_couples = remove_no_match(remove_matched(all_couples))
couples_df = pd.DataFrame(data=remaining_couples, columns=["man","woman"])
couples_df["probability"] = np.empty((len(couples_df), 0)).tolist()

In [337]:
def append_probability (df) :
    for row in df.itertuples(index=False)  :
        for couple in couples_df.itertuples(index=False)  :
            if row.man == couple.man :
                if row.woman == couple.woman :
                    couple.probability.append(row.probability)
                else :
                    continue
            else :
                continue

In [338]:
for n in range(1,11) :
    exec("append_probability(match{n}_df)".format(n=n))

In [339]:
def average (df_list):
    if len(df_list["probability"]) == 0 :
        _ = 0
    else :
        _ = sum(df_list["probability"])/ len(df_list["probability"])
    return _

In [340]:
couples_df["total_probability"] = couples_df.apply(average, axis=1)

In [341]:
couples_df

,man,woman,probability,total_probability
0,Anthony,Briana,[0.5],0.500
1,Anthony,Christina,[],0.000
2,Anthony,Tyler,[],0.000
3,Anthony,Alexandria,"[1.0, 0.75]",0.875
4,Nathan,Ellie,[0.75],0.750
5,Nathan,Tyler,[],0.000
6,Brandon,Briana,[0.75],0.750
7,Layton,Christina,[],0.000
8,Layton,Tyler,[],0.000
9,Layton,Alexandria,[],0.000


In [342]:
couples_df.sort_values(by=["woman"])

,man,woman,probability,total_probability
9,Layton,Alexandria,[],0.000
3,Anthony,Alexandria,"[1.0, 0.75]",0.875
18,Dario,Ashley,[],0.000
6,Brandon,Briana,[0.75],0.750
11,Garland,Briana,[],0.000
0,Anthony,Briana,[0.5],0.500
1,Anthony,Christina,[],0.000
7,Layton,Christina,[],0.000
12,Garland,Christina,[],0.000
4,Nathan,Ellie,[0.75],0.750
